In [553]:
import ee
import folium  # Interactive mapping
from datetime import datetime, timezone
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt




ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AVG7fiS84HxrdTl5K_IhK3YiVEn7iSEj_ZZcSvPxhFbRlblnPg9tRyvpS-s

Successfully saved authorization token.


In [658]:
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)  # Scale by 10000 to convert to reflectance values

def apply_cloud_mask(s2_img):
    # Extract date from Sentinel-2 image ID
    s2_img_id = s2_img.id()
    s2_date = ee.Date(s2_img.get('system:time_start'))

    # Filter Cloud Score+ collection for images on the same date as the Sentinel-2 image
    cloud_score_img = csPlus.filterDate(s2_date.format('YYYY-MM-dd'), s2_date.advance(1, 'day').format('YYYY-MM-dd')) \
                            .filter(ee.Filter.eq('system:index', s2_img_id)) \
                            .first()

    # Check if a corresponding cloud score image was found
    if cloud_score_img:
        cloud_mask = cloud_score_img.select('cs_cdf').gte(0.60)
        return s2_img.updateMask(cloud_mask)
    else:
        return s2_img  # Return the S2 image unmodified if no corresponding CS+ image is found

#AOI_sites: NE1,NE2,NE3,NE4,CE1,CE2,CE3,CE4,CE5,CE6,CE7,SE1,SE2,SE3,SE4.
    
aoi_test="CE7"

#aoi = ee.FeatureCollection('projects/ee-xalekter/assets/ee_bio/test_yarselja')
aoi = ee.FeatureCollection('projects/ee-xalekter/assets/ee_bio/est_'+aoi_test)

countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017');
aoi2 = countries.filter(ee.Filter.eq('country_na', 'Estonia'));


csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
QA_BAND = 'cs_cdf'
CLEAR_THRESHOLD = 0.60
# Filter Sentinel-2 ImageCollection for the day
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate('2024-04-01', '2024-04-30') \
        .filterBounds(aoi) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60))\
        .map(apply_cloud_mask)
 

medianComposite = collection.mean().clip(aoi)

print('Collection size: ', collection.size().getInfo())

Collection size:  15


In [657]:
visualization = {
    'min': 0.0,
    'max': 3000.0,
    'bands': ['B4', 'B3', 'B2'],
}
import folium

# Function to add Earth Engine image tiles to Folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data Â© Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.Map
folium.Map.add_ee_layer = add_ee_layer

# Define a Folium map with a given center and zoom level
my_map = folium.Map(location=[58.4326, 26.1872], zoom_start=9)

# Add the Sentinel-2 image layer to the map
my_map.add_ee_layer(medianComposite, visualization, 'Sentinel-2 Composite')

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
my_map

In [653]:
medianComposite = medianComposite.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])

# Reproject the image to EPSG:32635
reprojectedImage = medianComposite.reproject(crs='EPSG:32635', scale=10)

# Export the reprojected image
task = ee.batch.Export.image.toDrive(
    image=reprojectedImage,
    description='Sentinel2_est24_04_'+aoi_test,
    folder='GEE_exports',
    fileNamePrefix='S2_est_24_04_'+aoi_test,
    region=aoi.geometry(),
    scale=10,
    fileFormat='GeoTIFF'
)
task.start()